
### Welcome to mathématical planning.




# Techno used :

Mainly based on the llm through the libraries langchain, langgraph and langsmith.
At first we'll use a notebook to configure and test the design of the differents parts of the workflow.
Next, we'll build the workflow mith langgraph
Then, we'll use "deep_note" and streamlit to make the final tool

## Load the API keys

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")

# 0- Init the flow

First I will work with the most classic openai, gpt-4o-mini. I love the ollama deepseek-r1 mais il est si lent à tourner en local.
Peut être à faire évoluer vers du cloud.



In [2]:
# Installation des dépendances nécessaires
# pip install langchain langchain-community pypdf

# Chemin vers votre fichier PDF
pdf_path = "./data_test/proba.pdf"


# 1- load the sources pdf 

## 1-1 Load the pdf course

In [3]:
import pdfplumber

def extract_text_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Utilisation
pdf_text = extract_text_pdfplumber(pdf_path)
print(pdf_text)


Table des matières
1 Dénombrement . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.1 Permutations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.2 Arrangements . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.3 Combinaisons . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 2
1.4 Principes additif et multiplicatif . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
2 Introduction aux probabilités . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7
2.1 Généralités . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 8
2.2 Axiomes . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 10
2.3 Equiprobabilité . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11
3 Probabilités conditionnelles . . . . . . . . . . . . . . . . . . . . . . .

### 1-2 Extraction du chapitre

In [4]:
from flow_tools.nodes import model_query
from langchain_openai import ChatOpenAI

nom_chapitre = "3 Probabilités conditionnelles"
question_template = """Peux tu extraire le chapitre {nom_chapitre} du texte suivant : 
    {pdf_content}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chapitre = model_query(llm=llm, prompt_template=question_template, inputs={"nom_chapitre":nom_chapitre, "pdf_content":pdf_text})

In [5]:
print(chapitre)

### 3 PROBABILITÉS CONDITIONNELLES

**DÉFINITION 25.**  
Parfois il est utile de supposer qu'un événement a nécessairement eu lieu. Dans ce cas, on peut s'intéresser à la probabilité que A ait eu lieu sachant que B a eu lieu. On note cette probabilité \( P(A|B) \) et on a  
\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

**Exemple 26.**  
On veut calculer la probabilité qu'un jet de dé équilibré donne au moins un 2 sachant que le résultat est inférieur ou égal à 5. On définit alors deux événements A: «obtenir ...» et B: «obtenir ...» et on a \( P(A|B) = ... \)

**DÉFINITION 27.**  
Deux événements sont dits indépendants si  
\[
P(A \cap B) = P(A)P(B)
\]  
ce qui est équivalent à  
\[
P(A|B) = P(A)
\]  
On admet que si A et B sont indépendants, alors A et B le sont aussi.

**Exemple 28.**  
On tire une boule parmi 15 boules numérotées de 1 à 15. Les événements «le numéro tiré est pair» et «le numéro tiré est un multiple de 3» sont-ils indépendants?

**DÉFINITION 29.**  
On appelle partition de

### 1-3 Extraction des definitions

In [7]:
question_template = """Peux tu extraire les définitions du chapitre suivant : 
    {chapitre}
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

definitions = model_query(llm=llm, prompt_template=question_template, inputs={"chapitre":chapitre})



In [8]:
print(definitions)

Voici les définitions extraites du chapitre sur les probabilités conditionnelles :

### DÉFINITION 25
La probabilité d'un événement \( A \) sachant qu'un événement \( B \) a eu lieu est notée \( P(A|B) \) et est définie par la formule :
\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

### DÉFINITION 27
Deux événements \( A \) et \( B \) sont dits indépendants si :
\[
P(A \cap B) = P(A)P(B)
\]
Cela est équivalent à dire que :
\[
P(A|B) = P(A)
\]
On admet que si \( A \) et \( B \) sont indépendants, alors \( B \) et \( A \) le sont aussi.

### DÉFINITION 29
Une partition de \( \Omega \) est un ensemble d'événements \( (A_1, \ldots, A_k) \) tel que :
1. Les événements \( (A_i) \) sont disjoints : \( \forall i,j \in \llbracket 1;k \rrbracket, A_i \cap A_j = \emptyset \)
2. Les événements \( (A_i) \) recouvrent \( \Omega \) : \( A_1 \cup A_2 \cup \ldots \cup A_k = \Omega \)

### THÉORÈME 30
Soit \( (A_1, \ldots, A_k) \) une partition de \( \Omega \) avec \( \forall i \in \llbracket 1;k \rrbracket, 

### DÉFINITION 25
La probabilité d'un événement \( A \) sachant qu'un événement \( B \) a eu lieu est notée \( P(A|B) \) et est définie par la formule :
\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

### DÉFINITION 27
Deux événements \( A \) et \( B \) sont dits indépendants si :
\[
P(A \cap B) = P(A)P(B)
\]
Cela est équivalent à dire que :
\[
P(A|B) = P(A)
\]
On admet que si \( A \) et \( B \) sont indépendants, alors \( B \) et \( A \) le sont aussi.

### DÉFINITION 29
Une partition de \( \Omega \) est un ensemble d'événements \( (A_1, \ldots, A_k) \) tel que :
1. Les événements \( (A_i) \) sont disjoints : \( \forall i,j \in \llbracket 1;k \rrbracket, A_i \cap A_j = \emptyset \)
2. Les événements \( (A_i) \) recouvrent \( \Omega \) : \( A_1 \cup A_2 \cup \ldots \cup A_k = \Omega \)

### THÉORÈME 30
Soit \( (A_1, \ldots, A_k) \) une partition de \( \Omega \) avec \( \forall i \in \llbracket 1;k \rrbracket, P(A_i) \neq 0 \) et \( B \) un événement d'un espace probabilisé \( \Omega \). Alors :
\[
P(B) = \sum_{i=1}^{k} P(A_i \cap B) = \sum_{i=1}^{k} P(B|A_i)P(A_i)
\]


In [ ]:
### DÉFINITION 25
La probabilité d'un événement \( A \) sachant qu'un événement \( B \) a eu lieu est notée \( P(A|B) \) et est définie par la formule :
\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

### DÉFINITION 27
Deux événements \( A \) et \( B \) sont dits indépendants si :
\[
P(A \cap B) = P(A)P(B)
\]
Cela est équivalent à dire que :
\[
P(A|B) = P(A)
\]
On admet que si \( A \) et \( B \) sont indépendants, alors \( B \) et \( A \) le sont aussi.

### DÉFINITION 29
Une partition de \( \Omega \) est un ensemble d'événements \( (A_1, \ldots, A_k) \) tel que :
1. Les événements \( (A_i) \) sont disjoints : \( \forall i,j \in \llbracket 1;k \rrbracket, A_i \cap A_j = \emptyset \)
2. Les événements \( (A_i) \) recouvrent \( \Omega \) : \( A_1 \cup A_2 \cup \ldots \cup A_k = \Omega \)

### THÉORÈME 30
Soit \( (A_1, \ldots, A_k) \) une partition de \( \Omega \) avec \( \forall i \in \llbracket 1;k \rrbracket, P(A_i) \neq 0 \) et \( B \) un événement d'un espace probabilisé \( \Omega \). Alors :
\[
P(B) = \sum_{i=1}^{k} P(A_i \cap B) = \sum_{i=1}^{k} P(B|A_i)P(A_i)
\]


## 2- Illustration des définitions

In [16]:
question_template = """A partir de chacune des définitions et théorèmes suivants : 
    {definitions}
    peux tu créer des enrichir ces définitions sans les modifier en leur ajoutant des : 
    ## Illustrations
    ## Exercice corrigés
    permettant de les illustrer sequentiellement en t'appuyant sur le contenu du chapitre :
    {chapitre} 
    ?
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

definitions_illustrated = model_query(llm=llm, prompt_template=question_template, inputs={"definitions":definitions, "chapitre":chapitre})

In [17]:
print(definitions_illustrated)

Voici les enrichissements des définitions et théorèmes avec des illustrations et des exercices corrigés :

### DÉFINITION 25
La probabilité d'un événement \( A \) sachant qu'un événement \( B \) a eu lieu est notée \( P(A|B) \) et est définie par la formule :
\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

#### Illustrations
Imaginons un sac contenant 5 billes rouges et 3 billes bleues. Si nous savons qu'une bille tirée est rouge, la probabilité de tirer une bille rouge sachant qu'une bille a été tirée est \( P(A|B) \), où \( A \) est l'événement "tirer une bille rouge" et \( B \) est l'événement "tirer une bille".

#### Exercice corrigé
**Exemple 26.**  
On veut calculer la probabilité qu'un jet de dé équilibré donne au moins un 2 sachant que le résultat est inférieur ou égal à 5.  
- Soit \( A \) l'événement "obtenir au moins un 2" et \( B \) l'événement "obtenir un nombre inférieur ou égal à 5".  
- Les résultats possibles pour \( B \) sont {1, 2, 3, 4, 5}.  
- L'événement \( A \cap B \) c